# Importing Important Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from sklearn import preprocessing

In [2]:
DATA_PATH = '../data'
MODEL_PATH = '../src/models'

# Install Cleaned Dataset

In [3]:
df = pd.read_parquet(f'{DATA_PATH}/processed/data_clean.pqt')
df.head()

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category,year,month,day
2,0,acc_0,TST CASA DEL RIO EXP FAIRLAWN OH,18.42,2022-09-26,FOOD_AND_BEVERAGES,2022,9,26
4,0,acc_0,BUFFALO WILD WINGS,26.47,2022-09-12,FOOD_AND_BEVERAGES,2022,9,12
6,0,acc_0,OCULUS CA,11.73,2022-04-18,GENERAL_MERCHANDISE,2022,4,18
7,0,acc_0,LOS GIRASOLES STOW OH,30.04,2022-03-09,FOOD_AND_BEVERAGES,2022,3,9
8,0,acc_0,BUZZIS LAUNDRY OH,4.16,2022-03-29,GENERAL_MERCHANDISE,2022,3,29


# Preprocess Data for Category Categorization

In [4]:
data = df[['category', 'memo', 'amount', 'year', 'month', 'day']].copy()
data.head()

,category,memo,amount,year,month,day
2,FOOD_AND_BEVERAGES,TST CASA DEL RIO EXP FAIRLAWN OH,18.42,2022,9,26
4,FOOD_AND_BEVERAGES,BUFFALO WILD WINGS,26.47,2022,9,12
6,GENERAL_MERCHANDISE,OCULUS CA,11.73,2022,4,18
7,FOOD_AND_BEVERAGES,LOS GIRASOLES STOW OH,30.04,2022,3,9
8,GENERAL_MERCHANDISE,BUZZIS LAUNDRY OH,4.16,2022,3,29


In [5]:
categories = data['category'].unique()
categories.sort()
cat_dict = dict(zip(categories, np.arange(len(categories))))
cat_dict

{'EDUCATION': 0,
 'FOOD_AND_BEVERAGES': 1,
 'GENERAL_MERCHANDISE': 2,
 'GROCERIES': 3,
 'MORTGAGE': 4,
 'OVERDRAFT': 5,
 'PETS': 6,
 'RENT': 7,
 'TRAVEL': 8}

In [6]:
data['category_label'] = data['category'].map(cat_dict)
data.head()

,category,memo,amount,year,month,day,category_label
2,FOOD_AND_BEVERAGES,TST CASA DEL RIO EXP FAIRLAWN OH,18.42,2022,9,26,1
4,FOOD_AND_BEVERAGES,BUFFALO WILD WINGS,26.47,2022,9,12,1
6,GENERAL_MERCHANDISE,OCULUS CA,11.73,2022,4,18,2
7,FOOD_AND_BEVERAGES,LOS GIRASOLES STOW OH,30.04,2022,3,9,1
8,GENERAL_MERCHANDISE,BUZZIS LAUNDRY OH,4.16,2022,3,29,2


In [25]:
X_train, X_test, y_train, y_test = train_test_split(data[['memo']], data['category_label'], test_size=0.2, random_state=707)

In [26]:
train_data = X_train.merge(y_train, left_index=True, right_index=True)

train_balanced = train_data.groupby('category_label').apply(lambda x: x.sample(50000, replace=True)).reset_index(drop=True)
train_balanced.head()

,memo,category_label
0,PURCHASE AUTHORIZED ON UNIVERSITY OF DENV HIGH...,0
1,OFFICE MA W COLLE,0
2,NBS FACTS PYMT PLAN FE,0
3,RVT * ANAHUAC ISD EMILY ADLONG,0
4,WITHDRAWAL VISA CHECK CARD SCHOLASTIC EDUCATIO...,0


In [27]:
# X_train, X_valid, y_train, y_valid = train_test_split(train_balanced[['memo']], train_balanced['category_label'], test_size=0.2, random_state=707)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=707)

# Data Scaling Preprocessing and TF-IDF Vectorization

In [28]:
vectorizer = TfidfVectorizer()
vectorizer.fit(np.array(train_data['memo']))

TfidfVectorizer()

In [29]:
X_train = vectorizer.transform(np.array(X_train['memo']))
y_train = np.array(y_train)

X_valid = vectorizer.transform(np.array(X_valid['memo']))
y_valid = np.array(y_valid)

X_test = vectorizer.transform(np.array(X_test['memo']))
y_test = np.array(y_test)

In [30]:
# scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)

# X_train = scaler.transform(X_train)
# X_valid = scaler.transform(X_valid)
# X_test = scaler.transform(X_test)

# Baseline Memo Categorization

In [31]:
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

/Users/darrenjiang/opt/anaconda3/envs/dsc180/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
print(f"Training Accuracy: {sum(model.predict(X_train) == y_train) / y_train.shape[0]}")
print(f"Validation Accuracy: {sum(model.predict(X_valid) == y_valid) / y_valid.shape[0]}")
print(f"Training Accuracy: {sum(model.predict(X_test) == y_test) / y_test.shape[0]}")

Training Accuracy: 0.976508381491829
Validation Accuracy: 0.9724493261829472
Training Accuracy: 0.9728425395440333
